In [1]:
import numpy as np 
import os 
import cv2
from random import shuffle
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_dir = "train"
test_dir = "test1"

In [3]:
epochs=10
batch_size=64
img_size = 64
lr = 1e-4

In [4]:
def label_img(img):
	word_label = img.split('.')[-3]

	if word_label == "cat":
		return 1

	return 0

In [5]:
def create_train_data():
	training_data=[]
	for img in tqdm(os.listdir(train_dir)):
		label = label_img(img)
		img_path = os.path.join(train_dir,img)
		img = cv2.imread(img_path)
		img = cv2.resize(img, (img_size,img_size))
		training_data.append([np.array(img),label])

	shuffle(training_data)
	np.save('training_data.npy',training_data)

	return training_data

In [6]:
def create_test_data():
	testing_data=[]
	for img in tqdm(os.listdir(test_dir)):
		img_path = os.path.join(test_dir,img)
		img_num = img.split('.')[0]
		img = cv2.imread(img_path)
		img = cv2.resize(img, (img_size,img_size))
		testing_data.append([np.array(img), img_num])

	shuffle(testing_data)
	np.save('testing_data.npy',testing_data)

	return testing_data

In [7]:
train_data = np.load('training_data.npy')

In [8]:
train = train_data[:-500]
test = train_data[-500:]
train.shape

(24500, 2)

In [9]:
X = np.array([i[0]/255 for i in train]).reshape(-1,img_size,img_size,3)
X = X.flatten().reshape(len(X),img_size*img_size*3)
print(X.shape)
Y = np.array([i[1] for i in train])

(24500, 12288)


In [10]:
test_x = np.array([i[0]/255 for i in test]).reshape(-1,img_size,img_size,3)
test_x = test_x.flatten().reshape(len(test_x),img_size*img_size*3)
test_y = [i[1] for i in test]

In [11]:
model=Sequential()
model.add(Dense(256,activation='relu',input_dim=img_size*img_size*3,name="kauzi1"))
model.add(Dense(128,activation='relu',input_dim=img_size*img_size,name="kauzi2"))
model.add(Dense(64,activation='relu',input_dim=img_size*img_size,name="kauzi3"))
model.add(Dense(32,activation='relu',name="kauzi4"))
model.add(Dense(1,activation='sigmoid',name="kauzi5"))

In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
kauzi1 (Dense)               (None, 256)               3145984   
_________________________________________________________________
kauzi2 (Dense)               (None, 128)               32896     
_________________________________________________________________
kauzi3 (Dense)               (None, 64)                8256      
_________________________________________________________________
kauzi4 (Dense)               (None, 32)                2080      
_________________________________________________________________
kauzi5 (Dense)               (None, 1)                 33        
Total params: 3,189,249
Trainable params: 3,189,249
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
model.fit(X,Y,epochs=epochs,batch_size=batch_size,verbose = 1,validation_data=(test_x,test_y))

Train on 24500 samples, validate on 500 samples
Epoch 1/10
24500/24500 [==============================] - 17s 700us/step - loss: 0.7062 - acc: 0.5605 - val_loss: 0.6992 - val_acc: 0.5660
Epoch 2/10
24500/24500 [==============================] - 16s 655us/step - loss: 0.6680 - acc: 0.5938 - val_loss: 0.6459 - val_acc: 0.6080
Epoch 3/10
24500/24500 [==============================] - 16s 655us/step - loss: 0.6549 - acc: 0.6111 - val_loss: 0.6946 - val_acc: 0.5840
Epoch 4/10
24500/24500 [==============================] - 16s 655us/step - loss: 0.6483 - acc: 0.6156 - val_loss: 0.6630 - val_acc: 0.6280
Epoch 5/10
24500/24500 [==============================] - 16s 655us/step - loss: 0.6449 - acc: 0.6245 - val_loss: 0.6435 - val_acc: 0.6140
Epoch 6/10
24500/24500 [==============================] - 16s 657us/step - loss: 0.6347 - acc: 0.6364 - val_loss: 0.6697 - val_acc: 0.5980
Epoch 7/10
24500/24500 [==============================] - 16s 651us/step - loss: 0.6338 - acc: 0.6302 - val_loss: 0.63

In [15]:
img=test_x[2].reshape(64,64,3)
cv2.namedWindow('image',cv2.WINDOW_NORMAL)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
pred=model.predict(np.array([test_x[2],]))
if pred>=0 and pred<=0.5 :
    print("dog")
else :
    print("cat")

cat
